# Network Intrusion Detection with Deep Learning

In [0]:
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3" #model will be trained on GPU 0

In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

## The Data

In [0]:
import numpy as np
def load_matrices(file_name):
    with open(file_name, 'rb') as f:
        A = np.load(f)
        B = np.load(f)
        C = np.load(f)
        D = np.load(f)
        E = np.load(f)
        F = np.load(f)
    return (A,B,C,D,E,F)

In [0]:
my_file =  '10data2.npy'
train,test,y_train,y_test,ylabel_train, ylabel_test = load_matrices(my_file)

## The Model

In [0]:
from keras import backend as K
#from keras.layers import Dense, Activation, Merge, Reshape, Dropout
from keras.layers import Activation, Reshape, Input, Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.layers import InputLayer, Input
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, EarlyStopping
from keras.models import load_model
from keras.models import Model, Sequential
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.advanced_activations import PReLU
from keras.activations import elu
from keras.losses import binary_crossentropy

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

In [0]:
import talos as ta
import wrangle as wr
from talos.metrics.keras_metrics import fmeasure_acc
from talos import live



In [0]:
def selu(x):
    """Scaled Exponential Linear Unit. (Klambauer et al., 2017)
    # Arguments
        x: A tensor or variable to compute the activation function for.
    # References
        - [Self-Normalizing Neural Networks](https://arxiv.org/abs/1706.02515)
    """
    alpha = 1.6732632423543772848170429916717
    scale = 1.0507009873554804934193349852946
    return scale * elu(x, alpha)

#model.add(Dense(32, input_shape=(input_length - 1,)), activation=selu)

In [0]:
#actvation leakyrelu
LR= LeakyReLU(0.01)
LR.__name__ = 'lrelu'
#actvation prelu
#PR= PReLU(init='zero', weights=None)
#PR.__name__ = 'prelu'

In [0]:
from keras.callbacks import ModelCheckpoint, CSVLogger
#filepath="./logtalos/talosdnn_weights_a.hdf"
#checkpoint = ModelCheckpoint(filepath, monitor='val_loss', save_best_only=True, mode='min')
#csv_logger = CSVLogger('./logtalos/autoencoder.csv',separator=',', append=False)
#callbacks_list = [checkpoint,csv_logger]

In [0]:
def log_file_name1(first_neuron,second_neuron,third_neuron, fourth_neuron,batch_size,kernel_initial,lr,activation):
    
    if activation==LR:
       activation= 'lrelu'
    
    if activation==PReLU:
       activation= 'prelu'
    
    if activation==relu:
       activation= 'relu'
    
    if activation==elu:
       activation= 'elu'
    
    if activation==selu:
       activation= 'selu'

    # The dir-name for the TensorBoard log-dir.
    s = "./logtalos/log7lrelu/neuron1_{0}-neuron2_{1}-neuron3_{2}-neuron4_{3}-batch{4}-kernelinit_{5}-lr{6}-activ_{7}.hdf"
    log_file = s.format(first_neuron,second_neuron,third_neuron,fourth_neuron,batch_size,kernel_initial,lr,activation)
    return log_file

def log_file_name2(first_neuron,second_neuron,third_neuron, fourth_neuron,batch_size,kernel_initial,lr,activation):
    
    if activation==LR:
       activation= 'lrelu'
    
    if activation==PReLU:
       activation= 'prelu'
    
    if activation==relu:
       activation= 'relu'
    
    if activation==elu:
       activation= 'elu'
    
    if activation==selu:
       activation= 'selu'
    
    # The dir-name for the TensorBoard log-dir.
    s = "./logtalos/log7lrelu/neuron1_{0}-neuron2_{1}-neuron3_{2}-neuron4_{3}-batch{4}-kernelinit_{5}-lr{6}-activ_{7}.csv"
    log_file = s.format(first_neuron,second_neuron,third_neuron, fourth_neuron, batch_size,kernel_initial,lr,activation)
    return log_file


In [0]:
#from sklearn.utils.class_weight import compute_class_weight

In [0]:
#class_weight = compute_class_weight(class_weight='balanced',
#                                    classes=np.unique(target_class),
#                                    y=target_class)

In [0]:
#class_weight

In [0]:
from keras.callbacks import ModelCheckpoint
filepath="./logtalos/7layertes_lrelu.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [0]:
from keras.models import Model, Sequential

In [0]:
# first we have to make sure to input data and params into the function
# add input parameters to the function
def dnn(x_train, ylabel_train, x_val, y_val, params):

    input_dim = x_train.shape[1]

    input_img = Input(shape=(input_dim,))
    
    first_neuron=params['first_neuron']
    second_neuron=params['second_neuron']
    third_neuron=params['third_neuron']
    fourth_neuron=params['fourth_neuron']
    batch_size=params['batch_size']
    kernel_initial=params['kernel_initializer']
    lr=params['lr']
    activation=params['activation']
    
    print('activation=',activation)
    
    log_file1 = log_file_name1(first_neuron,second_neuron,third_neuron,fourth_neuron, batch_size,kernel_initial,lr,activation)
    checkpoint2 = ModelCheckpoint(log_file1, monitor='acc', save_best_only=True, mode='max')

    
    log_file2 = log_file_name2(first_neuron,second_neuron,third_neuron,fourth_neuron,batch_size,kernel_initial,lr,activation)

    csv_logger2 = CSVLogger(log_file2,separator=',', append=False)
    callbacks_list2 = [checkpoint2,csv_logger2]
    
    ###AE
    
    # replace the hyperparameter inputs with references to params dictionary 
    autoencoder = Sequential()
    #encoder
    
    if activation==PReLU: 
        autoencoder.add(Dense(first_neuron, input_shape=(input_dim,), kernel_initializer=kernel_initial))
        autoencoder.add(PReLU())
        autoencoder.add(Dense(second_neuron, kernel_initializer=kernel_initial))
        autoencoder.add(PReLU())
        autoencoder.add(Dense(third_neuron, kernel_initializer=kernel_initial))
        autoencoder.add(PReLU())
        autoencoder.add(Dense(fourth_neuron, kernel_initializer=kernel_initial))
        autoencoder.add(PReLU())
                   
        #decoder
        #model.add(Dense(8, activation=params['activation']))
        autoencoder.add(Dense(third_neuron, kernel_initializer=kernel_initial))
        autoencoder.add(PReLU())
        autoencoder.add(Dense(second_neuron, kernel_initializer=kernel_initial))
        autoencoder.add(PReLU())
        autoencoder.add(Dense(first_neuron, kernel_initializer=kernel_initial))
        autoencoder.add(PReLU())
        autoencoder.add(Dense(input_dim, activation='sigmoid', kernel_initializer=kernel_initial))
        
    else:
        autoencoder.add(Dense(first_neuron, input_shape=(input_dim,), activation=activation, kernel_initializer=kernel_initial))
        autoencoder.add(Dense(second_neuron, activation=activation, kernel_initializer=kernel_initial))
        autoencoder.add(Dense(third_neuron, activation=activation, kernel_initializer=kernel_initial))
        autoencoder.add(Dense(fourth_neuron, activation=activation, kernel_initializer=kernel_initial))
        #decoder
        #model.add(Dense(8, activation=params['activation']))
        autoencoder.add(Dense(third_neuron, activation=activation, kernel_initializer=kernel_initial))
        autoencoder.add(Dense(second_neuron, activation=activation, kernel_initializer=kernel_initial))
        autoencoder.add(Dense(first_neuron, activation=activation, kernel_initializer=kernel_initial))
        autoencoder.add(Dense(input_dim, activation='sigmoid', kernel_initializer=kernel_initial))
    
    #model = multi_gpu_model(model, gpus=1)
    autoencoder.summary()
    print(input_dim)
    
    input_img = Input(shape=(input_dim,))
    encoder_layer1 = autoencoder.layers[0]
    encoder_layer2 = autoencoder.layers[1]
    encoder_layer3 = autoencoder.layers[2]
    encoder_layer4 = autoencoder.layers[3]
    encoder = Model(input_img, encoder_layer4(encoder_layer3(encoder_layer2(encoder_layer1(input_img)))))
    
    #encoder_layer1 = autoencoder.layers[0]
    #encoder_layer2 = autoencoder.layers[1]
    #encoder_layer3 = autoencoder.layers[2]
    #encoder = Model(input_img, encoder_layer3(encoder_layer2(encoder_layer1(input_img))))
    encoder.summary()

    
    autoencoder.compile(loss='mse', optimizer='adam', metrics=['acc'])
    #model.compile(loss=params['losses'],
    #              optimizer=params['optimizer'](),
    #              metrics=['acc', fmeasure_acc])
    
   

    
    history=autoencoder.fit(x,x,
                             epochs=params['epochs'],
                             batch_size=batch_size,
                              #shuffle=True,
                              validation_split=0.2,
                              callbacks=callbacks_list,                    
                             #callbacks=[early_stopping])
                              #validation_data=(test,test),
                              verbose=0)
  

    num_classes=15
    out2 = Dense(num_classes, activation='softmax',kernel_initializer=kernel_initial)(encoder.output)
    newmodel = Model(encoder.input,out2)
    
   
    newmodel.compile(loss='categorical_crossentropy', 
                     #optimizer='adam', 
                     optimizer= Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08,decay=0.0, amsgrad=False),
                     metrics=['acc']) 
    newmodel.summary()


    history2a=newmodel.fit(x, y,
                           epochs=params['epochs2'],
                           batch_size=batch_size,
                           validation_split=0.2,
                           callbacks=callbacks_list2,
                           #callbacks=[early_stopping],
                           #validation_data=(test,y_test),
                           verbose=0)

    
    del encoder
    del autoencoder
    #del dnn_model
    
    tf.reset_default_graph()
    
    return history2a, newmodel #, encoder
    

In [0]:
# then we can go ahead and set the parameter space
p = {'first_neuron':[70,60],
     'second_neuron':[50,40],
     'third_neuron':[35,30],
     'fourth_neuron':[25,20],
     'batch_size': [32,64,256],
     'epochs': [20],
     'epochs2': [30],
     'dropout': [0],
     'kernel_initializer': ['lecun_uniform','lecun_normal','he_normal','he_uniform','normal','glorot_uniform','glorot_normal'],
     'optimizer': [Adam],
     'lr':[0.01, 0.001,0.0001],#0.00001],
     #'activation':[LR]}#, elu,selu]}
     'activation':[relu, elu,selu,  LR, PReLU]}

In [0]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session, clear_session

config = tf.ConfigProto(allow_soft_placement=True)
#config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

In [0]:
x=train
y=ylabel_train

In [0]:
np.shape(x)

(1619315, 80)

In [0]:
np.shape(y)

(1619315, 15)

In [0]:

t = ta.Scan(x, y, p, dnn, experiment_no='71',grid_downsample=0.05,)


  0%|          | 0/50 [00:00<?, ?it/s]

activation= <keras.layers.advanced_activations.LeakyReLU object at 0x7efbf9389ba8>
Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 60)                4860      
_________________________________________________________________
dense_2 (Dense)              (None, 40)                2440      
_________________________________________________________________
dense_3 (Dense)              (None, 35)                1435      
_________________________________________________________________
dense_4 (Dense)              (None, 20)                720       
_________________________________________________________________
dense_5 (Dense)              (None, 35)                735       
_________________________________________________________________
dense_6 (Dense)              (N

  2%|▏         | 1/50 [34:24<28:06:12, 2064.74s/it]

activation= <keras.layers.advanced_activations.LeakyReLU object at 0x7efbf9389ba8>
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 60)                4860      
_________________________________________________________________
dense_2 (Dense)              (None, 50)                3050      
_________________________________________________________________
dense_3 (Dense)              (None, 35)                1785      
_________________________________________________________________
dense_4 (Dense)              (None, 20)                720       
_________________________________________________________________
dense_5 (Dense)              (None, 35)                735       
_________________________________________________________________
dense_6 (Dense)              (None, 50)                1800      
_____________________________________

  4%|▍         | 2/50 [1:09:14<27:37:41, 2072.11s/it]

...

activation= <keras.layers.advanced_activations.LeakyReLU object at 0x7efbf9389ba8>
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 70)                5670      
_________________________________________________________________
dense_2 (Dense)              (None, 50)                3550      
_________________________________________________________________
dense_3 (Dense)              (None, 35)                1785      
_________________________________________________________________
dense_4 (Dense)              (None, 20)                720       
_________________________________________________________________
dense_5 (Dense)              (None, 35)                735       
_________________________________________________________________
dense_6 (Dense)              (None, 50)                1800      
_____________________________________

 50%|█████     | 25/50 [16:49:40<19:34:47, 2819.51s/it]

activation= <keras.layers.advanced_activations.LeakyReLU object at 0x7efbf9389ba8>
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 60)                4860      
_________________________________________________________________
dense_2 (Dense)              (None, 50)                3050      
_________________________________________________________________
dense_3 (Dense)              (None, 35)                1785      
_________________________________________________________________
dense_4 (Dense)              (None, 20)                720       
_________________________________________________________________
dense_5 (Dense)              (None, 35)                735       
_________________________________________________________________
dense_6 (Dense)              (None, 50)                1800      
_____________________________________

 52%|█████▏    | 26/50 [17:43:48<19:39:10, 2947.92s/it]

activation= <keras.layers.advanced_activations.LeakyReLU object at 0x7efbf9389ba8>
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 70)                5670      
_________________________________________________________________
dense_2 (Dense)              (None, 50)                3550      
_________________________________________________________________
dense_3 (Dense)              (None, 35)                1785      
_________________________________________________________________
dense_4 (Dense)              (None, 20)                720       
_________________________________________________________________
dense_5 (Dense)              (None, 35)                735       
_________________________________________________________________
dense_6 (Dense)              (None, 50)                1800      
_____________________________________

In [0]:
x.shape

In [0]:
t.data

In [0]:
r = ta.Reporting(t)

In [0]:
r.best_params()